<a href="https://colab.research.google.com/github/daawonn/ml_dl_projects/blob/master/2_%EA%B0%84%EB%8B%A8%ED%95%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B0%8F_%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C%EC%9A%B0%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%98%EC%97%AC_%EC%98%88%EC%B8%A1%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 연구목표

- BMI 지수 = (몸무게)/(키*키)
- BMI를 이용하여 비만도를 측정한다
- 알고리즘 svm을 이용하여 BMI를 학습, 비만도 판정
- 데이터
  - 3_bmi.csv
  - 업로드하여 사용

# 데이터 획득
 - 개발환경이 colab으로 변경되어서 로컬파일을 업로드하여 사용해야 될 것 같다.
 

In [0]:
import numpy as np
import pandas as pd

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 3_bmi.csv to 3_bmi (1).csv
User uploaded file "3_bmi.csv" with length 264371 bytes


In [0]:
df = pd.read_csv('3_bmi.csv')
df.head(2)

,height,weight,label
0,178,69,normal
1,190,62,thin


# 데이터 준비

- 품질 향상

In [0]:
# 데이터의 크기, 컬럼(피쳐의 종류), 레이블
df.shape, df.columns, df['label'].unique()

((20000, 3),
 Index(['height', 'weight', 'label'], dtype='object'),
 array(['normal', 'thin', 'fat'], dtype=object))

In [0]:
df.dtypes

height     int64
weight     int64
label     object
dtype: object

In [0]:
df['height'].max(), df['height'].min(), df['weight'].max(), df['weight'].min()

(200, 120, 80, 35)

In [0]:
# 데이터의 정규화 -> 무게, 키의 최대치를 기준으로 처리하시오
# 처리된 값을 다시 해당 컬럼에 대입한다.
df['height'] = df['height']/df['height'].max()
df['weight'] = df['weight']/df['weight'].max()
df.head(3)

,height,weight,label
0,0.89,0.8625,normal
1,0.95,0.7750,thin
2,0.67,0.7625,fat


In [0]:
# 레이블 : 종속변수  => 분류를 벡터화 해서 처리
# 케이스가 총 3개 (normal,thin,fat) => 001,010,100
# thin    => [1,0,0]
# normal  => [0,1,0]
# fat     => [0,0,1]
# 으로 값이 변경되도록 처리하시오

# np.array => 배열로 처리함

matchingTable = {
  'thin'   : np.array([1,0,0]),
  'normal' : np.array([0,1,0]),
  'fat'    : np.array([0,0,1])
}
matchingTable

df['label'] = df['label'].apply(lambda x:matchingTable[x])
df.head(3)

,height,weight,label
0,0.89,0.8625,"[0, 1, 0]"
1,0.95,0.7750,"[1, 0, 0]"
2,0.67,0.7625,"[0, 0, 1]"


In [0]:
# 연습
# df['label'] = df['label'].apply(lambda x: [1,0,0] if x=="thin" else x)
# df['label'] = df['label'].apply(lambda x: [0,1,0] if x=="normal" else x)
# df['label'] = df['label'].apply(lambda x: [0,0,1] if x=="fat" else x)
# df.head(3)

,height,weight,label
0,0.89,0.8625,"[0, 1, 0]"
1,0.95,0.7750,"[1, 0, 0]"
2,0.67,0.7625,"[0, 0, 1]"


In [0]:
# 데이터를 훈련용, 테스트용으로 분류한다
from sklearn.model_selection import train_test_split

In [0]:
# 비율 75:25 , 난수: 0 으로 고정하여 분류하시오
X = df[['height','weight']]
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.75, test_size=0.25,random_state=0 )

In [0]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((15000, 2), (5000, 2), (15000,), (5000,))

# 데이터 분석
- 생략

# 머신러닝 모델링(딥러닝, 텐서플로우 사용)

- 각각의 출력값에 대해서, 가중치를 곱하고 합한후, bias(편향, 조정한값)을 더해서 softmax라는 함수로 적용한것 (간단한 퍼셉트론 표현)
- 해당 이론을 수식으로 정리하여 함수로 표현
[그림 sm0, sm1, sm2, sm3 참조]
  - y = softmax(Wx + b)
- 해당 수식을 데이터 플로우 그래프로 표현  

In [0]:
import tensorflow as tf

In [0]:
# 텐서플로우로 이 수식을 처리하기 위해 구현
# 입력 x : 키, 몸무게 
x = tf.placeholder( tf.float32, [None,2] ) 

In [0]:
# 가중치 => 값이 특정되지 않고 바뀔수 있다 => 변수
# shape 형태를 고려
# tf.zeros()를 사용하면 초기값을 0으로 부여하여 전체 타입을 float32로 통일
W = tf.Variable( tf.zeros([2, 3]) )

In [0]:
# 편향 b : 변수
b = tf.Variable( tf.zeros([3]) )

In [0]:
# y = softmax(Wx + b)
# 데이터 플로우 그래프
# x는 matrix, 2차 행렬, [None , 2] 
# 행렬의 곱 [None , 2] * ( 2,3 ) => 최종 : ( None,3 )형태  # 3 => thin, normal, fat 
y = tf.nn.softmax(tf.matmul(x,W) + b)

In [0]:
# 데이터 플로우 그래프의 핵심 => shape 이 중요! 맞춰주기
'%s X %s + %s = %s' % (x.shape, W.shape, b.shape, y.shape)

'(?, 2) X (2, 3) + (3,) = (?, 3)'

# 시스템 통합
- 산출물 :skip